# Model - RandomForest with log of target

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Remove Outliers---

In [4]:
# OutlierRemover is a custom transformer imported from src/helpers.py
# It can remove outliers from multiple columns by specifying the iqr_multipler

# Features where there are significant outliers per EDA
outlier_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea',
                'OpenPorchSF']

# Set the iqr multiplier to 5, to remove only the extreme outliers, in order to keep as much data as possible
outlier_remover = OutlierRemover(columns=outlier_cols,iqr_multiplier=5)

# Fit the dataset to the outlier remover and save to new variable
outlier_remover.fit(df)
df_proc = outlier_remover.transform(df)

# Check how many observations remain
df_proc.count()

Id               1428
MSSubClass       1428
MSZoning         1428
LotFrontage      1178
LotArea          1428
                 ... 
MoSold           1428
YrSold           1428
SaleType         1428
SaleCondition    1428
SalePrice        1428
Length: 81, dtype: int64

# ---Pipeline---

### 1. Filter for features to remove

In [5]:
# Defining the list of columns that will go into the pipeline
# Remove columns that should not be included in the pipeline

# Drop SalePrice b/c target feature
# Drop Id b/c no predictive meaning
# Drop Street b/c low variance
# Drop Utilities b/c low variance
# Drop GarageYrBlt b/c imputing is too speculative

all_columns = df.columns.tolist()
columns_remove_from_filter = ['SalePrice','Id','Street','Utilities']

columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [6]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

### 2. Imputing null variables

In [7]:
# Preprocessor for imputing all null variables

cols_impute_na = ['Alley', 'GarageType', 'Fence','BsmtQual', 'BsmtCond', 'BsmtExposure',
                  'BsmtFinType1','BsmtFinType2', 'FireplaceQu', 'GarageFinish', 'GarageQual','GarageCond',
                  'PoolQC', 'MiscFeature']
cols_impute_none = ['MasVnrType']
cols_impute_most_freq = ['Electrical','MSZoning','Exterior1st','Exterior2nd','KitchenQual','Functional']
cols_impute_zero = ['LotFrontage', 'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath',
                    'GarageCars','GarageArea','GarageYrBlt']

preprocessor_imputer = ColumnTransformer([
  ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),
  ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),
  ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),
  ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)
], remainder='passthrough', verbose_feature_names_out=False)

### 3. Feature Engineering

In [8]:
# create a custom transformer to incorporate the engineered features into the pipeline

class FeatureEngineer(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, X):
    X = X.copy()

    X['RatioBathBed'] = (X['FullBath'].fillna(0)+X['HalfBath'].fillna(0)) / (X['BedroomAbvGr'].fillna(0)+1)
    X['HouseAge'] = X['YrSold'] - X['YearBuilt']
    X['TotalBaths'] = X['FullBath'].fillna(0) + (X['HalfBath'].fillna(0)*0.5) + X['BsmtFullBath'].fillna(0) + (X['BsmtHalfBath'].fillna(0)*0.5)
    X['FireBedRatio'] = X['Fireplaces'].fillna(0) / (X['BedroomAbvGr'].fillna(0) + 1)

    return X


engineer = FeatureEngineer()
df_engineered = engineer.transform(df_proc)

### 4. Mapping Ordinal Features

In [9]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

### 5. Encoding and Scaling

In [10]:
standard_cols = ['LotFrontage','GarageArea']

robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
               'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal', 'LowQualFinSF',
               '3SsnPorch']

minmax_cols = ['YearBuilt','YearRemodAdd']


preprocessor_scalers = ColumnTransformer([
  ('standard_scaler', StandardScaler(), standard_cols),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), minmax_cols)
], remainder='passthrough', verbose_feature_names_out=False)

scaler_pipe = Pipeline([
  ('preprocessor_scalers', preprocessor_scalers),
])


In [11]:
# Preprocessor for encoding and scaling

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']


preprocessor_encode_scale = ColumnTransformer([
  ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),
  ('oe', OrdinalEncoder(mapping=ordinal_cat_map), oe_cols)],
  remainder=scaler_pipe, verbose_feature_names_out=False)

# --- Train/Val Split, Fit & Transform ---

In [12]:
# Train/Val split, target feature is SalePrice

X = df.drop(columns='SalePrice')
y = df['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# take the log of the target to normalize the distribution

log_y_train = np.log(y_train)
log_y_val = np.log(y_val)

In [14]:
# Main pipeline, include all preprocessors

main_pipe = Pipeline([
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('engineer', engineer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [15]:
lasso_pipe = Pipeline([
  ('main_pipe', main_pipe),
  ('lasso', Lasso(random_state=42))
])

In [16]:
lasso_pipe.get_params()

{'memory': None,
 'steps': [('main_pipe',
   Pipeline(steps=[('preprocessor_filter',
                    ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                     ['MSSubClass', 'MSZoning',
                                                      'LotFrontage', 'LotArea',
                                                      'Alley', 'LotShape',
                                                      'LandContour', 'LotConfig',
                                                      'LandSlope', 'Neighborhood',
                                                      'Condition1', 'Condition2',
                                                      'BldgType', 'HouseStyle',
                                                      'OverallQual', 'OverallCond',
                                                      'YearBuilt', 'YearRemodAdd',
                                                      'RoofStyle', 'RoofMatl',
                        

# --- Model Evaluation ---

In [17]:
from sklearn.model_selection import GridSearchCV

params = {
  'lasso__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
  'lasso__max_iter': [1000, 5000, 10000]
}

lasso_gs = GridSearchCV(lasso_pipe, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

lasso_gs.fit(X_train, log_y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/Users/annahan/Documents/GitHub/learning-fuze/live_class_W1D2_0125/.conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.535e-02, tolerance: 1.428e-02
  model = cd_fast.enet_coordinate_descent(
/Users/annahan/Documents/GitHub/learning-fuze/live_class_W1D2_0125/.conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.308e-01, tolerance: 1.436e-02
  model = cd_fast.enet_coordinate_descent(
/Users/annahan/Documents/GitHub/learning-fuze/live_class_W1D2_0125/.conda/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Obje

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('main_pipe',
                                        Pipeline(steps=[('preprocessor_filter',
                                                         ColumnTransformer(transformers=[('passthrough',
                                                                                          'passthrough',
                                                                                          ['MSSubClass',
                                                                                           'MSZoning',
                                                                                           'LotFrontage',
                                                                                           'LotArea',
                                                                                           'Alley',
                                                                                           'LotShape',
                                                                                           'LandContour',
                                                                                           'LotConfig',
                                                                                           'LandSlope',
                                                                                           'Neighborhood',
                                                                                           'Condition1',
                                                                                           'Condition2',
                                                                                           'BldgType',
                                                                                           'HouseStyle',
                                                                                           'OverallQual',
                                                                                           'Overall...
                                                                                           'BsmtFinType1',
                                                                                           'BsmtFinType2',
                                                                                           'HeatingQC',
                                                                                           'FireplaceQu',
                                                                                           'GarageFinish',
                                                                                           'GarageQual',
                                                                                           'GarageCond',
                                                                                           'PoolQC',
                                                                                           'KitchenQual',
                                                                                           'Fence'])],
                                                                           verbose_feature_names_out=False))])),
                                       ('lasso', Lasso(random_state=42))]),
             n_jobs=-1,
             param_grid={'lasso__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                         'lasso__max_iter': [1000, 5000, 10000]},
             scoring='neg_mean_squared_error', verbose=1)

In [18]:
lasso_gs.best_params_

{'lasso__alpha': 0.001, 'lasso__max_iter': 1000}

In [19]:
results_train = evaluate_regression(lasso_gs, X_train, log_y_train, 'lasso_gs')
results_val = evaluate_regression(lasso_gs, X_val, log_y_val, 'lasso_gs')

In [20]:
results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
lasso_gs,0.080172,0.014123,26885.98102,0.006697,0.907355,0.900537


In [21]:
results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
lasso_gs,0.092378,0.018557,25760.883035,0.007782,0.900558,0.862855


In [22]:
# predict val
log_val_preds = lasso_gs.predict(X_val)

# convert prediction back by using exponential
val_preds = np.exp(log_val_preds)

# calculate log RMSE
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.1362247152416538

In [23]:
model_index = lasso_gs.best_estimator_.named_steps['lasso'].feature_names_in_

In [24]:
model_coef = lasso_gs.best_estimator_.named_steps['lasso'].coef_

In [25]:
pd.set_option('display.max_rows', 300)

feat_coef = pd.Series(model_coef, index=model_index)
feat_coef.abs().sort_values(ascending=False)


RoofMatl_ClyTile         2.963498e-01
GrLivArea                1.224599e-01
Neighborhood_StoneBr     8.976374e-02
Neighborhood_Crawfor     8.470562e-02
Neighborhood_NridgHt     8.206356e-02
Exterior1st_BrkFace      7.802719e-02
MSSubClass_160           7.420669e-02
OverallQual              6.086540e-02
GarageCars               5.456538e-02
Neighborhood_Somerst     4.796369e-02
MSZoning_RM              4.785066e-02
Condition1_Norm          4.675989e-02
Functional_Typ           4.345488e-02
SaleCondition_Abnorml    3.973763e-02
BldgType_1Fam            3.836333e-02
CentralAir_N             3.822192e-02
OverallCond              3.803569e-02
TotalBaths               3.777577e-02
MSSubClass_20            3.671002e-02
LotConfig_CulDSac        3.582933e-02
LandContour_Bnk          3.479796e-02
MSSubClass_30            3.142488e-02
Foundation_PConc         3.122717e-02
Neighborhood_Edwards     3.112070e-02
Neighborhood_NoRidge     3.103711e-02
Neighborhood_BrkSide     2.890191e-02
BsmtQual    

# --- Predicting test ---

In [26]:
# import the test data

test_data = pd.read_csv('../data/test.csv')

In [27]:
# save the id column to be added back later to the predictions

test_id = test_data.loc[:,'Id'].copy()
test_id = pd.DataFrame(test_id)

In [28]:
# predict test (keeping in mind it is still log)

log_test_pred = lasso_gs.predict(test_data)
log_test_pred = pd.DataFrame(log_test_pred)

In [29]:
# convert the test predictions back using exponential

test_pred = np.exp(log_test_pred)

In [30]:
# add the id's back to the test predictions
# change the column names to match the submission format
# check the final submission

combined_test = test_id.join(test_pred)
combined_test.columns = ['Id','SalePrice']
combined_test

,Id,SalePrice
0,1461,113787.626625
1,1462,144600.584791
2,1463,169364.573711
3,1464,194587.799173
4,1465,199000.291945
...,...,...
1454,2915,86968.641687
1455,2916,87540.371461
1456,2917,159580.815709
1457,2918,118189.007126


In [31]:
# save results to csv

combined_test.to_csv('../data/test_preds6_lasso_gs.csv', index=False)

In [32]:
# save results to csv - submission

combined_test.to_csv('../data/submission6.csv', index=False)

In [33]:
# calculate kaggle ranking, what percentage I am in

929/3661

0.2537558044250205